# Data Story

1. Questions
    - Comparisons
    - Counts
    - Trends
    - Plots (Bar/Histogram/Scatter/Time-Series)
    - Cross-tabs
2. Insights
    - Correlations
    - Hypotheses
3. Narrative
    - Present in a story form, needs to flow
    - What trends/relationships would make it more complete?

**Notes:**

- Date/Time needs to be categorized before being used in ML models
- Only choose a few columns to explore in detail, can always go back and explore further after starting ML
- *Histogram of diff in appointment-date relative to scheduled-date*